## Étude suggestion métier vers code rome
l'API matching utilisait jusqu'à présent l'API LBB pour la suggestion métier / code rome.

Vu le besoin en performances accrus, le moment est venu d'intégrer cette fonctionnalite dans l'API matching. Ce notebook sert de base de travail à cet effet.

### Améliorations:
- V accent folding & nettoyage pour recherche normalisée
- V drop_duplicates pour éviter les retours mulitples identiques
- Ajout score ordre / tri
- Utiliser une librairie NLP (Spacy, NLTK, ...)

### Notes:
Nettoyage INPUT: https://machinelearningmastery.com/clean-text-machine-learning-python/

In [2]:
## Chargement des labels rome, ogr
from IPython.display import display, HTML
import pandas as pd
import numpy as np

# Rome
rome_labels = pd.read_csv('/ressources/liste_rome_LBB.csv', sep=',', encoding="utf-8")
rome_labels.columns = ['rome', 'rome_1', 'rome_2', 'rome_3', 'label', 'slug']
print(f"Obtained {len(rome_labels)} ROME labels")
display(HTML(rome_labels.head(5).to_html()))

# OGR
ogr_labels = pd.read_csv('/ressources/list_OGR_LBB.csv', sep=',', encoding="utf-8")
ogr_labels.columns = ['code', 'rome_1', 'rome_2', 'rome_3', 'label', 'rome']
print(f"Obtained {len(ogr_labels)} OGR labels")
display(HTML(ogr_labels.head(5).to_html()))


Obtained 531 ROME labels


,rome,rome_1,rome_2,rome_3,label,slug
0,A1101,A,11,1,Conduite d'engins agricoles et forestiers,conduite-d-engins-agricoles-et-forestiers
1,A1201,A,12,1,Bûcheronnage et élagage,bucheronnage-et-elagage
2,A1202,A,12,2,Entretien des espaces naturels,entretien-des-espaces-naturels
3,A1203,A,12,3,Entretien des espaces verts,entretien-des-espaces-verts
4,A1204,A,12,4,Protection du patrimoine naturel,protection-du-patrimoine-naturel


Obtained 10948 OGR labels


,code,rome_1,rome_2,rome_3,label,rome
0,11987,A,11,1,Chauffeur / Chauffeuse de machines agricoles,A1101
1,12862,A,11,1,Conducteur / Conductrice d'abatteuses,A1101
2,38874,A,11,1,Conducteur / Conductrice d'automoteur de récolte,A1101
3,13232,A,11,1,Conducteur / Conductrice de machines à vendanger,A1101
4,38878,A,11,1,Conducteur / Conductrice de matériels de semis,A1101


In [3]:
import string
import unidecode
def normalize(txt):
    # Remove punctuation
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    txt = txt.translate(table)
    
    # Lowercase
    txt = txt.lower()
    
    # Remove short letter groups
    txt = txt.split()
    txt = [t for t in txt if len(t) >= 3]
    txt = ' '.join(txt)
    
    # Accent folding
    txt = unidecode.unidecode(txt)
    
    return txt

rome_labels['stack'] = rome_labels.apply(lambda x: normalize(x['label']), axis=1)
ogr_labels['stack'] = ogr_labels.apply(lambda x: normalize(x['label']), axis=1)

In [109]:
import string
from fuzzywuzzy import fuzz
import re

def words_get(raw_query):
    query = normalize(raw_query)
    return query.split()

def result_build(score, rome, rome_label, rome_slug, ogr_label=None):
    if ogr_label:
        label = f"{rome_label} ({ogr_label}, ...)"
    else:
        label = rome_label
    return {
        'id': rome,
        'label': label,
        'value': ogr_label if ogr_label else rome_label,
        'occupation': rome_slug,
        'score': score
    }

def check_full_query(query):
    """
    Correct suggestion when full job title has been entered
    """
    title = re.sub(r'\([^\)]*\)', '', query).strip()
    is_full = False
    if ' ' in title:
        is_full = True
    return is_full, normalize(title)

def score_build(query, match):
    ratio = fuzz.ratio(query, match)
    # ratio scores on 100, we reduce to 5 with 1 decimal
    return(round(ratio / 20, 1))

def rome_suggest(query, rome_df, ogr_df):
    results = {}
    
    # Unelegant solution to uncontroled queries
    is_full, needle = check_full_query(query)

    check = rome_df[rome_df['stack'] == needle]
    if not check.empty:
        # Full title match, only one result possible
        print(check.iloc[0]['rome'])
        results[check.iloc[0]['rome']] = result_build(
            100,
            check.iloc[0]['rome'],
            check.iloc[0]['label'],
            check.iloc[0]['slug']
        )
        return results.values()

    check = rome_df[rome_df['stack'].str.contains(needle)]
    if not check.empty:
        results[check.iloc[0]['rome']] = result_build(
                score_build(needle, check.iloc[0]['stack']),
                check.iloc[0]['rome'],
                check.iloc[0]['label'],
                check.iloc[0]['slug']
            )

    words = words_get(query)
    rome_raw_matches = []
    ogr_raw_matches = []
    # Only using first 5 words
    for word in words[:5]:
        rome_raw_matches.append( rome_df[rome_df['stack'].str.contains(word)] )
        ogr_raw_matches.append( ogr_df[ogr_df['stack'].str.contains(word)] )
    rome_matches = pd.concat(rome_raw_matches)
    ogr_matches = pd.concat(ogr_raw_matches)
    
    for _i, ogr_row in ogr_matches.iterrows():
        rome = ogr_row['rome']
        ogr_romes = rome_df[rome_df['rome'] == rome]
        for _j, rome_row in ogr_romes.iterrows():
            results[rome] = result_build(
                score_build(query, ogr_row['stack']),
                rome,
                rome_row['label'],
                rome_row['slug'],
                ogr_row['label']
            )
    
    for _i, rome_row in rome_matches.iterrows():
        results[rome_row['rome']] = result_build(
            score_build(query, rome_row['stack']),
            rome_row['rome'],
            rome_row['label'],
            rome_row['slug']
        )
        
    return sorted(list(results.values()), key=lambda e: e['score'], reverse=True)
    

## Test Suggestion

In [120]:
## TEST SUGGESTIOn
#recherche = "journa"
#recherche = "Vente en articles de sport et loisirs (Vendeur / Vendeuse de journaux, ...)"
recherche = "dresseur animaux"
##
res = rome_suggest(recherche, rome_labels, ogr_labels)
res

[{'id': 'L1204',
  'label': "Arts du cirque et arts visuels (Dresseur / Dresseuse d'animaux de cirque, ...)",
  'value': "Dresseur / Dresseuse d'animaux de cirque",
  'occupation': 'arts-du-cirque-et-arts-visuels',
  'score': 3.2},
 {'id': 'D1210',
  'label': "Vente en animalerie (Vendeur / Vendeuse d'animaux, ...)",
  'value': "Vendeur / Vendeuse d'animaux",
  'occupation': 'vente-en-animalerie',
  'score': 3.2},
 {'id': 'A1501',
  'label': 'Aide aux soins animaux',
  'value': 'Aide aux soins animaux',
  'occupation': 'aide-aux-soins-animaux',
  'score': 3.1},
 {'id': 'A1503',
  'label': 'Toilettage des animaux',
  'value': 'Toilettage des animaux',
  'occupation': 'toilettage-des-animaux',
  'score': 2.9},
 {'id': 'H2414',
  'label': "Préparation et finition d'articles en cuir et matériaux souples (Dresseur / Dresseuse en ganterie, ...)",
  'value': 'Dresseur / Dresseuse en ganterie',
  'occupation': 'preparation-et-finition-d-articles-en-cuir-et-materiaux-souples',
  'score': 2.8},


In [102]:
# Test scoring fuzz
from fuzzywuzzy import fuzz
Str1 = "Apple Inc."
Str2 = "apple Inc"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
print(Ratio)

query = "philos"
for r in res:
    ratio = fuzz.partial_ratio(query, normalize(r['label']))
    print(f'{query} in {r["value"]} score: {ratio}')


95
philos in Coiffeur barbier / Coiffeuse barbière score: 33


In [17]:
# test requête
query="Phil"
q=f'.*{query}.*'

res = ogr_labels[ogr_labels['label'].str.contains(query)]
results = []
for index, row in res.iterrows():
    rome = row['rome']
    label = row['label']
    romes = rome_labels[rome_labels['rome'] == rome]
    for index, rome_row in romes.iterrows():
        results.append({
            'code': rome,
            'label': f"{rome_row['label']} ({label}, ...)"
        })
results

[{'code': 'K2401',
  'label': "Recherche en sciences de l'homme et de la société (Philologue, ...)"}]

In [7]:
query="phil"
ogr_labels[ogr_labels['label'].str.contains(query)]

,code,rome_1,rome_2,rome_3,label,rome,stack
1493,38630,D,12,10,Vendeur / Vendeuse conseil en aquariophilie,D1210,vendeur vendeuse conseil aquariophilie
8385,38346,K,21,7,Professeur / Professeure de philosophie,K2107,professeur professeure philosophie
8895,10370,K,25,3,Agent / Agente cynophile de sécurité,K2503,agent agente cynophile securite
